# MODEL-7: Uncertainty Quantification for RUL Prediction

This notebook implements uncertainty quantification techniques for Remaining Useful Life (RUL) prediction:

1. **Gaussian Output Layer** using `tfp.layers.DistributionLambda` for aleatoric uncertainty
2. **Negative Log-Likelihood Loss** for training distributional outputs
3. **Monte Carlo Dropout** for epistemic uncertainty estimation
4. **Prediction Intervals** visualization on RUL curves
5. **Calibration Analysis** to verify uncertainty quality

## Uncertainty Types

- **Aleatoric Uncertainty**: Irreducible noise in the data (captured by Gaussian output)
- **Epistemic Uncertainty**: Model uncertainty due to limited data (captured by MC Dropout)

## Architecture

```
Input (spectrograms or raw signals)
    ↓
Feature Extractor (CNN or TCN)
    ↓
Gaussian Output Layer (tfp.layers.DistributionLambda)
    ↓
Normal(μ, σ) → RUL prediction with uncertainty
```

In [ ]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Add project root to path
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# Use tf_keras for TensorFlow Probability compatibility
import tf_keras as keras
from tf_keras import layers

# Import TensorFlow Probability
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Probability version: {tfp.__version__}")

## 1. Load Data and Generate Spectrograms

In [ ]:
from src.data.loader import XJTUBearingLoader
from src.data.rul_labels import generate_rul_labels
from src.features.stft import extract_spectrogram

# Initialize loader
loader = XJTUBearingLoader()
metadata = loader.get_metadata()

print(f"Dataset overview:")
print(f"  Conditions: {list(metadata.keys())}")
print(f"  Total bearings: {sum(len(c) for c in metadata.values())}")

In [ ]:
# Load data for one bearing
condition = '35Hz12kN'
bearing_id = 'Bearing1_1'

signals, files = loader.load_bearing(condition, bearing_id)
print(f"{bearing_id}: {len(files)} files")

# Generate spectrograms
n_samples = len(signals)
print(f"Generating {n_samples} spectrograms...")

spectrograms = []
for i in range(n_samples):
    spec = extract_spectrogram(signals[i], sampling_rate=25600.0)
    spectrograms.append(spec)

X = np.stack(spectrograms).astype(np.float32)
print(f"Spectrograms shape: {X.shape}")

# Generate RUL labels (using string strategy, not enum)
y = generate_rul_labels(
    num_files=len(files),
    strategy='piecewise_linear',
    max_rul=125
).astype(np.float32)

print(f"RUL range: {y.min():.1f} - {y.max():.1f}")

In [ ]:
# Train/validation split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training: {X_train.shape[0]} samples")
print(f"Validation: {X_val.shape[0]} samples")

## 2. Gaussian Output Layer with TensorFlow Probability

We implement a Gaussian output layer using `tfp.layers.DistributionLambda` that outputs a Normal distribution.
The network learns both the mean (μ) and standard deviation (σ) of the RUL prediction.

In [ ]:
def build_gaussian_output_model(
    input_shape=(128, 128, 2),
    base_filters=32,
    num_blocks=4,
):
    """Build CNN model with Gaussian output layer for uncertainty quantification.
    
    The model outputs a Normal distribution N(μ, σ) where:
    - μ (mean) is the predicted RUL
    - σ (std) represents the aleatoric uncertainty
    
    Args:
        input_shape: Shape of input spectrograms.
        base_filters: Base number of filters for CNN.
        num_blocks: Number of convolutional blocks.
        
    Returns:
        Keras model outputting a TFP distribution.
    """
    inputs = keras.Input(shape=input_shape, name='spectrogram_input')
    
    # CNN backbone
    x = inputs
    for i in range(num_blocks):
        filters = base_filters * (2 ** i)
        x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(2)(x)
    
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    # Output 2 parameters: mean and log(variance)
    # Using log(variance) ensures variance is always positive after exp()
    params = layers.Dense(2, name='distribution_params')(x)
    
    # Create Gaussian distribution using DistributionLambda
    # The lambda splits params into mean and std
    distribution = tfpl.DistributionLambda(
        lambda t: tfd.Normal(
            loc=tf.nn.relu(t[..., :1]),  # Mean (non-negative via ReLU)
            scale=tf.nn.softplus(t[..., 1:]) + 1e-6  # Std (positive via softplus)
        ),
        name='gaussian_output'
    )(params)
    
    model = keras.Model(inputs=inputs, outputs=distribution, name='gaussian_rul_model')
    return model


# Build the model
gaussian_model = build_gaussian_output_model()
gaussian_model.summary()

## 3. Negative Log-Likelihood Loss

For distributional outputs, we use Negative Log-Likelihood (NLL) as the loss function:

$$\mathcal{L}_{NLL} = -\log p(y | \mu, \sigma) = \frac{1}{2}\log(2\pi\sigma^2) + \frac{(y - \mu)^2}{2\sigma^2}$$

This loss encourages the model to:
1. Predict accurate mean values (μ close to true y)
2. Estimate appropriate uncertainty (σ matches prediction error)

In [ ]:
def negative_log_likelihood(y_true, y_pred_distribution):
    """Negative log-likelihood loss for distributional output.
    
    Args:
        y_true: True RUL values, shape (batch, 1).
        y_pred_distribution: TFP distribution object.
        
    Returns:
        Scalar loss value.
    """
    return -tf.reduce_mean(y_pred_distribution.log_prob(y_true))


# Compile model with NLL loss
gaussian_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=negative_log_likelihood,
)

print("Model compiled with Negative Log-Likelihood loss")

In [ ]:
# Train the Gaussian output model
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        verbose=1
    ),
]

# Reshape y for training
y_train_reshaped = y_train.reshape(-1, 1)
y_val_reshaped = y_val.reshape(-1, 1)

history_gaussian = gaussian_model.fit(
    X_train, y_train_reshaped,
    validation_data=(X_val, y_val_reshaped),
    epochs=50,
    batch_size=8,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Plot training history
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(history_gaussian.history['loss'], label='Train NLL')
ax.plot(history_gaussian.history['val_loss'], label='Val NLL')
ax.set_xlabel('Epoch')
ax.set_ylabel('Negative Log-Likelihood')
ax.set_title('Gaussian Model Training History')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.savefig('../outputs/models/uncertainty_training_history.png', dpi=150)
plt.show()

## 4. Monte Carlo Dropout for Epistemic Uncertainty

Monte Carlo Dropout estimates epistemic uncertainty by:
1. Keeping dropout active during inference
2. Running multiple forward passes
3. Computing mean and variance of predictions

This captures model uncertainty due to limited training data.

In [ ]:
def build_mc_dropout_model(
    input_shape=(128, 128, 2),
    base_filters=32,
    num_blocks=4,
    dropout_rate=0.2,
):
    """Build CNN model with MC Dropout for epistemic uncertainty.
    
    Uses dropout layers that remain active during inference (training=True).
    
    Args:
        input_shape: Shape of input spectrograms.
        base_filters: Base number of filters for CNN.
        num_blocks: Number of convolutional blocks.
        dropout_rate: Dropout rate for uncertainty estimation.
        
    Returns:
        Keras model with dropout.
    """
    inputs = keras.Input(shape=input_shape, name='spectrogram_input')
    
    # CNN backbone with dropout after each block
    x = inputs
    for i in range(num_blocks):
        filters = base_filters * (2 ** i)
        x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Dropout(dropout_rate)(x)  # Spatial dropout
    
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers with dropout
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # Output (non-negative RUL)
    outputs = layers.Dense(1, activation='relu', name='rul_output')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='mc_dropout_model')
    return model


def mc_dropout_predict(model, X, n_samples=50):
    """Make predictions with MC Dropout uncertainty estimation.
    
    Args:
        model: Trained Keras model with dropout layers.
        X: Input data.
        n_samples: Number of stochastic forward passes.
        
    Returns:
        mean: Mean prediction.
        std: Standard deviation (epistemic uncertainty).
        all_preds: All individual predictions.
    """
    predictions = []
    
    for _ in range(n_samples):
        # Set training=True to keep dropout active
        pred = model(X, training=True)
        predictions.append(pred.numpy())
    
    all_preds = np.stack(predictions, axis=0)  # (n_samples, batch, 1)
    mean = np.mean(all_preds, axis=0)
    std = np.std(all_preds, axis=0)
    
    return mean, std, all_preds


# Build MC Dropout model
mc_model = build_mc_dropout_model(dropout_rate=0.3)
mc_model.summary()

In [ ]:
# Train MC Dropout model
from src.training.config import TrainingConfig, compile_model

config = TrainingConfig(
    epochs=50,
    batch_size=8,
)
compile_model(mc_model, config)

history_mc = mc_model.fit(
    X_train, y_train_reshaped,
    validation_data=(X_val, y_val_reshaped),
    epochs=config.epochs,
    batch_size=config.batch_size,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# MC Dropout predictions
print("Running MC Dropout inference (50 forward passes)...")
mc_mean, mc_std, mc_all_preds = mc_dropout_predict(mc_model, X_val, n_samples=50)

print(f"MC Dropout Results:")
print(f"  Mean prediction shape: {mc_mean.shape}")
print(f"  Std (uncertainty) shape: {mc_std.shape}")
print(f"  Average uncertainty: {mc_std.mean():.4f}")

## 5. Visualize Prediction Intervals on RUL Curves

We visualize the predictions with uncertainty bands to show:
- Mean prediction (solid line)
- 68% confidence interval (1σ band)
- 95% confidence interval (2σ band)

In [ ]:
# Get Gaussian model predictions
gaussian_dist = gaussian_model(X_val)
gaussian_mean = gaussian_dist.mean().numpy()
gaussian_std = gaussian_dist.stddev().numpy()

print(f"Gaussian Model Results:")
print(f"  Mean prediction shape: {gaussian_mean.shape}")
print(f"  Std (aleatoric) shape: {gaussian_std.shape}")
print(f"  Average uncertainty: {gaussian_std.mean():.4f}")

In [ ]:
def plot_rul_with_uncertainty(
    y_true, y_pred_mean, y_pred_std, 
    title='RUL Prediction with Uncertainty',
    ax=None
):
    """Plot RUL predictions with uncertainty bands.
    
    Args:
        y_true: True RUL values.
        y_pred_mean: Predicted mean RUL.
        y_pred_std: Predicted standard deviation.
        title: Plot title.
        ax: Matplotlib axis (creates new if None).
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 6))
    
    # Sort by true RUL for cleaner visualization
    sort_idx = np.argsort(y_true.flatten())
    y_true_sorted = y_true.flatten()[sort_idx]
    y_mean_sorted = y_pred_mean.flatten()[sort_idx]
    y_std_sorted = y_pred_std.flatten()[sort_idx]
    
    x = np.arange(len(y_true_sorted))
    
    # Plot ground truth
    ax.plot(x, y_true_sorted, 'k-', linewidth=2, label='Ground Truth', alpha=0.8)
    
    # Plot mean prediction
    ax.plot(x, y_mean_sorted, 'b-', linewidth=2, label='Predicted Mean')
    
    # 68% confidence interval (1σ)
    ax.fill_between(
        x,
        y_mean_sorted - y_std_sorted,
        y_mean_sorted + y_std_sorted,
        alpha=0.3,
        color='blue',
        label='68% CI (1σ)'
    )
    
    # 95% confidence interval (2σ)
    ax.fill_between(
        x,
        y_mean_sorted - 1.96 * y_std_sorted,
        y_mean_sorted + 1.96 * y_std_sorted,
        alpha=0.15,
        color='blue',
        label='95% CI (2σ)'
    )
    
    ax.set_xlabel('Sample Index (sorted by true RUL)')
    ax.set_ylabel('RUL')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    return ax


# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_rul_with_uncertainty(
    y_val_reshaped, gaussian_mean, gaussian_std,
    title='Gaussian Output Model (Aleatoric Uncertainty)',
    ax=axes[0]
)

plot_rul_with_uncertainty(
    y_val_reshaped, mc_mean, mc_std,
    title='MC Dropout Model (Epistemic Uncertainty)',
    ax=axes[1]
)

plt.tight_layout()
plt.savefig('../outputs/models/uncertainty_prediction_intervals.png', dpi=150)
plt.show()

In [ ]:
# Plot uncertainty vs RUL position (expectation: higher uncertainty near end-of-life)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gaussian model
axes[0].scatter(y_val_reshaped, gaussian_std, alpha=0.6, c='blue')
axes[0].set_xlabel('True RUL')
axes[0].set_ylabel('Predicted Std (Uncertainty)')
axes[0].set_title('Gaussian Model: Uncertainty vs RUL')
axes[0].grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(y_val_reshaped.flatten(), gaussian_std.flatten(), 1)
p = np.poly1d(z)
x_line = np.linspace(y_val.min(), y_val.max(), 100)
axes[0].plot(x_line, p(x_line), 'r--', linewidth=2, label=f'Trend (slope={z[0]:.4f})')
axes[0].legend()

# MC Dropout model
axes[1].scatter(y_val_reshaped, mc_std, alpha=0.6, c='green')
axes[1].set_xlabel('True RUL')
axes[1].set_ylabel('MC Dropout Std (Uncertainty)')
axes[1].set_title('MC Dropout Model: Uncertainty vs RUL')
axes[1].grid(True, alpha=0.3)

# Add trend line
z2 = np.polyfit(y_val_reshaped.flatten(), mc_std.flatten(), 1)
p2 = np.poly1d(z2)
axes[1].plot(x_line, p2(x_line), 'r--', linewidth=2, label=f'Trend (slope={z2[0]:.4f})')
axes[1].legend()

plt.tight_layout()
plt.savefig('../outputs/models/uncertainty_vs_rul.png', dpi=150)
plt.show()

print(f"\nUncertainty Trend Analysis:")
print(f"  Gaussian: slope = {z[0]:.4f} (negative = higher uncertainty at low RUL)")
print(f"  MC Dropout: slope = {z2[0]:.4f}")

## 6. Calibration Analysis

A well-calibrated uncertainty estimate should have:
- ~68% of true values within 1σ interval
- ~95% of true values within 2σ interval

We plot a calibration curve comparing expected vs. observed coverage.

In [ ]:
def compute_calibration_curve(y_true, y_pred_mean, y_pred_std, n_bins=10):
    """Compute calibration curve for uncertainty estimates.
    
    Args:
        y_true: True values.
        y_pred_mean: Predicted mean.
        y_pred_std: Predicted standard deviation.
        n_bins: Number of confidence levels to evaluate.
        
    Returns:
        expected_coverage: Expected coverage percentages.
        observed_coverage: Observed coverage percentages.
    """
    # Confidence levels from 0.1 to 0.99
    confidence_levels = np.linspace(0.1, 0.99, n_bins)
    
    y_true_flat = y_true.flatten()
    y_mean_flat = y_pred_mean.flatten()
    y_std_flat = y_pred_std.flatten()
    
    observed_coverage = []
    
    for conf in confidence_levels:
        # Calculate z-score for this confidence level
        from scipy import stats
        z = stats.norm.ppf((1 + conf) / 2)
        
        # Check what fraction of true values fall within the interval
        lower = y_mean_flat - z * y_std_flat
        upper = y_mean_flat + z * y_std_flat
        
        within_interval = (y_true_flat >= lower) & (y_true_flat <= upper)
        observed = np.mean(within_interval)
        observed_coverage.append(observed)
    
    return confidence_levels, np.array(observed_coverage)


# Compute calibration for both models
conf_levels, obs_gaussian = compute_calibration_curve(
    y_val_reshaped, gaussian_mean, gaussian_std, n_bins=15
)
_, obs_mc = compute_calibration_curve(
    y_val_reshaped, mc_mean, mc_std, n_bins=15
)

print("Calibration Analysis Complete")

In [ ]:
# Plot calibration curves
fig, ax = plt.subplots(figsize=(8, 8))

# Perfect calibration line
ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect Calibration')

# Gaussian model calibration
ax.plot(conf_levels, obs_gaussian, 'b-o', linewidth=2, markersize=6, 
        label='Gaussian Output')

# MC Dropout calibration
ax.plot(conf_levels, obs_mc, 'g-s', linewidth=2, markersize=6,
        label='MC Dropout')

ax.set_xlabel('Expected Coverage (Confidence Level)', fontsize=12)
ax.set_ylabel('Observed Coverage', fontsize=12)
ax.set_title('Uncertainty Calibration Plot', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')

# Add calibration error annotation
ece_gaussian = np.mean(np.abs(conf_levels - obs_gaussian))
ece_mc = np.mean(np.abs(conf_levels - obs_mc))

ax.text(0.05, 0.90, f'ECE (Gaussian): {ece_gaussian:.3f}', transform=ax.transAxes, fontsize=11)
ax.text(0.05, 0.85, f'ECE (MC Dropout): {ece_mc:.3f}', transform=ax.transAxes, fontsize=11)

plt.tight_layout()
plt.savefig('../outputs/models/uncertainty_calibration.png', dpi=150)
plt.show()

print(f"\nExpected Calibration Error (ECE):")
print(f"  Gaussian Model: {ece_gaussian:.4f}")
print(f"  MC Dropout Model: {ece_mc:.4f}")
print(f"  (Lower ECE = better calibrated uncertainty)")

In [ ]:
# Check specific coverage rates
from scipy import stats

def check_coverage(y_true, y_mean, y_std, confidence_levels=[0.68, 0.95, 0.99]):
    """Check coverage at specific confidence levels."""
    results = []
    
    y_true_flat = y_true.flatten()
    y_mean_flat = y_mean.flatten()
    y_std_flat = y_std.flatten()
    
    for conf in confidence_levels:
        z = stats.norm.ppf((1 + conf) / 2)
        lower = y_mean_flat - z * y_std_flat
        upper = y_mean_flat + z * y_std_flat
        within = np.mean((y_true_flat >= lower) & (y_true_flat <= upper))
        results.append({
            'expected': conf,
            'observed': within,
            'gap': within - conf
        })
    
    return pd.DataFrame(results)

print("Gaussian Model Coverage:")
gaussian_coverage = check_coverage(y_val_reshaped, gaussian_mean, gaussian_std)
print(gaussian_coverage.to_string(index=False))

print("\nMC Dropout Model Coverage:")
mc_coverage = check_coverage(y_val_reshaped, mc_mean, mc_std)
print(mc_coverage.to_string(index=False))

## 7. Combined Uncertainty (Aleatoric + Epistemic)

For the most comprehensive uncertainty estimate, we can combine both types:

$$\sigma_{total}^2 = \sigma_{aleatoric}^2 + \sigma_{epistemic}^2$$

In [ ]:
def build_combined_uncertainty_model(
    input_shape=(128, 128, 2),
    base_filters=32,
    num_blocks=4,
    dropout_rate=0.2,
):
    """Build model that outputs Gaussian params with dropout for combined uncertainty.
    
    Combines:
    - Aleatoric uncertainty via Gaussian output (learned σ)
    - Epistemic uncertainty via MC Dropout (variance of predictions)
    """
    inputs = keras.Input(shape=input_shape, name='spectrogram_input')
    
    # CNN backbone with dropout
    x = inputs
    for i in range(num_blocks):
        filters = base_filters * (2 ** i)
        x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Dropout(dropout_rate)(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # Output distribution parameters
    params = layers.Dense(2, name='distribution_params')(x)
    
    # Gaussian distribution
    distribution = tfpl.DistributionLambda(
        lambda t: tfd.Normal(
            loc=tf.nn.relu(t[..., :1]),
            scale=tf.nn.softplus(t[..., 1:]) + 1e-6
        ),
        name='gaussian_output'
    )(params)
    
    model = keras.Model(inputs=inputs, outputs=distribution, name='combined_uncertainty_model')
    return model


def predict_with_combined_uncertainty(model, X, n_mc_samples=50):
    """Predict with combined aleatoric and epistemic uncertainty.
    
    Returns:
        mean: Mean prediction.
        aleatoric_std: Aleatoric uncertainty (average learned σ).
        epistemic_std: Epistemic uncertainty (std of means across MC samples).
        total_std: Combined uncertainty.
    """
    all_means = []
    all_stds = []
    
    for _ in range(n_mc_samples):
        dist = model(X, training=True)
        all_means.append(dist.mean().numpy())
        all_stds.append(dist.stddev().numpy())
    
    all_means = np.stack(all_means, axis=0)  # (n_mc, batch, 1)
    all_stds = np.stack(all_stds, axis=0)
    
    # Mean prediction
    mean = np.mean(all_means, axis=0)
    
    # Aleatoric uncertainty: average of learned stds
    aleatoric_std = np.mean(all_stds, axis=0)
    
    # Epistemic uncertainty: std of means across MC samples
    epistemic_std = np.std(all_means, axis=0)
    
    # Total uncertainty: sqrt(aleatoric^2 + epistemic^2)
    total_std = np.sqrt(aleatoric_std**2 + epistemic_std**2)
    
    return mean, aleatoric_std, epistemic_std, total_std


# Build and train combined model
combined_model = build_combined_uncertainty_model(dropout_rate=0.25)
combined_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=negative_log_likelihood,
)

print("Combined Uncertainty Model:")
combined_model.summary()

In [ ]:
# Train combined model
history_combined = combined_model.fit(
    X_train, y_train_reshaped,
    validation_data=(X_val, y_val_reshaped),
    epochs=50,
    batch_size=8,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Get combined uncertainty predictions
print("Computing combined uncertainty (50 MC samples)...")
comb_mean, comb_aleatoric, comb_epistemic, comb_total = predict_with_combined_uncertainty(
    combined_model, X_val, n_mc_samples=50
)

print(f"\nCombined Uncertainty Results:")
print(f"  Mean aleatoric uncertainty: {comb_aleatoric.mean():.4f}")
print(f"  Mean epistemic uncertainty: {comb_epistemic.mean():.4f}")
print(f"  Mean total uncertainty: {comb_total.mean():.4f}")

In [ ]:
# Visualize uncertainty decomposition
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Sort by true RUL
sort_idx = np.argsort(y_val_reshaped.flatten())
y_sorted = y_val_reshaped.flatten()[sort_idx]
aleatoric_sorted = comb_aleatoric.flatten()[sort_idx]
epistemic_sorted = comb_epistemic.flatten()[sort_idx]
total_sorted = comb_total.flatten()[sort_idx]

x = np.arange(len(y_sorted))

# Stacked area plot of uncertainties
axes[0].fill_between(x, 0, aleatoric_sorted, alpha=0.6, label='Aleatoric', color='blue')
axes[0].fill_between(x, aleatoric_sorted, aleatoric_sorted + epistemic_sorted, 
                     alpha=0.6, label='Epistemic', color='orange')
axes[0].set_xlabel('Sample Index (sorted by RUL)')
axes[0].set_ylabel('Uncertainty (Std)')
axes[0].set_title('Uncertainty Decomposition')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scatter: Aleatoric vs Epistemic
axes[1].scatter(comb_aleatoric, comb_epistemic, c=y_val_reshaped, cmap='coolwarm', alpha=0.7)
axes[1].set_xlabel('Aleatoric Uncertainty')
axes[1].set_ylabel('Epistemic Uncertainty')
axes[1].set_title('Aleatoric vs Epistemic (color=RUL)')
cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
cbar.set_label('True RUL')
axes[1].grid(True, alpha=0.3)

# Total uncertainty vs RUL
axes[2].scatter(y_val_reshaped, comb_total, alpha=0.6, c='purple')
axes[2].set_xlabel('True RUL')
axes[2].set_ylabel('Total Uncertainty')
axes[2].set_title('Total Uncertainty vs RUL')
axes[2].grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(y_val_reshaped.flatten(), comb_total.flatten(), 1)
p = np.poly1d(z)
x_line = np.linspace(y_val.min(), y_val.max(), 100)
axes[2].plot(x_line, p(x_line), 'r--', linewidth=2, label=f'Trend (slope={z[0]:.4f})')
axes[2].legend()

plt.tight_layout()
plt.savefig('../outputs/models/uncertainty_decomposition.png', dpi=150)
plt.show()

## 8. Metrics Comparison

In [ ]:
from src.training.metrics import rmse, mae, phm08_score

# Compute metrics for all models
results = []

# Gaussian model
results.append({
    'Model': 'Gaussian Output',
    'RMSE': rmse(y_val_reshaped.flatten(), gaussian_mean.flatten()),
    'MAE': mae(y_val_reshaped.flatten(), gaussian_mean.flatten()),
    'Mean Uncertainty': gaussian_std.mean(),
    'ECE': ece_gaussian,
})

# MC Dropout model
results.append({
    'Model': 'MC Dropout',
    'RMSE': rmse(y_val_reshaped.flatten(), mc_mean.flatten()),
    'MAE': mae(y_val_reshaped.flatten(), mc_mean.flatten()),
    'Mean Uncertainty': mc_std.mean(),
    'ECE': ece_mc,
})

# Combined model
_, obs_combined = compute_calibration_curve(
    y_val_reshaped, comb_mean, comb_total, n_bins=15
)
ece_combined = np.mean(np.abs(conf_levels - obs_combined))

results.append({
    'Model': 'Combined (Ale+Epi)',
    'RMSE': rmse(y_val_reshaped.flatten(), comb_mean.flatten()),
    'MAE': mae(y_val_reshaped.flatten(), comb_mean.flatten()),
    'Mean Uncertainty': comb_total.mean(),
    'ECE': ece_combined,
})

results_df = pd.DataFrame(results)
print("\n" + "="*70)
print("UNCERTAINTY QUANTIFICATION MODEL COMPARISON")
print("="*70)
print(results_df.to_string(index=False))
print("="*70)

# Save results
os.makedirs('../outputs/evaluation', exist_ok=True)
results_df.to_csv('../outputs/evaluation/uncertainty_comparison.csv', index=False)
print("\nResults saved to outputs/evaluation/uncertainty_comparison.csv")

## 9. Conclusions and Recommendations

### Key Findings

1. **Gaussian Output Layer** (tfp.layers.DistributionLambda)
   - Successfully learns aleatoric uncertainty alongside mean prediction
   - Trained with negative log-likelihood loss
   - Provides per-sample uncertainty estimates

2. **Monte Carlo Dropout**
   - Captures epistemic uncertainty via multiple stochastic forward passes
   - No architectural changes needed (just keep dropout active)
   - Computational cost scales with number of MC samples

3. **Uncertainty Behavior**
   - Expected: Higher uncertainty near end-of-life (low RUL)
   - Observed: Trend analysis shows uncertainty patterns across RUL range
   
4. **Calibration Quality**
   - Expected Calibration Error (ECE) measures reliability
   - Lower ECE indicates better-calibrated uncertainty
   - 68% CI should contain ~68% of true values

### Recommendations

1. **For Production**: Use combined uncertainty (aleatoric + epistemic)
2. **For Speed**: Use Gaussian output alone (single forward pass)
3. **For Robustness**: Use MC Dropout with 20-50 samples
4. **Always**: Validate calibration on held-out data

### Next Steps

1. Apply uncertainty models to full dataset with cross-validation
2. Integrate uncertainty into maintenance decision-making
3. Explore deep ensembles for additional diversity
4. Implement temperature scaling for calibration improvement

In [ ]:
# Save models
os.makedirs('../outputs/models', exist_ok=True)

gaussian_model.save('../outputs/models/gaussian_uncertainty_model.keras')
mc_model.save('../outputs/models/mc_dropout_model.keras')
combined_model.save('../outputs/models/combined_uncertainty_model.keras')

print("Models saved to outputs/models/")
print("  - gaussian_uncertainty_model.keras")
print("  - mc_dropout_model.keras")
print("  - combined_uncertainty_model.keras")